In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('./datasets/DfCleaned.csv')

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import LSTM, Dense
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

2023-11-10 05:51:02.061106: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 05:51:02.227198: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 05:51:02.227230: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 05:51:02.228246: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 05:51:02.317641: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 05:51:02.318402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Model n-gram2

In [5]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df['ngram2'].values)
x = tokenizer.texts_to_sequences(df['ngram2'].values)
x = pad_sequences(x)

In [6]:
#y = pd.get_dummies(df['clarity']).values
y = df['Class']

In [7]:
from sklearn.model_selection import KFold
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True)

In [8]:
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [11]:
embed_dim = 32
lstm_out = 32

model = Sequential([
    layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
    layers.Conv1D(filters=32, kernel_size=1, padding='same', activation= 'relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.LSTM(128, dropout=0.4, recurrent_dropout=0.1),
    layers.Dense(1, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3347, 32)          32000     
                                                                 
 conv1d_1 (Conv1D)           (None, 3347, 32)          1056      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 1673, 32)          0         
 g1D)                                                            
                                                                 
 lstm_1 (LSTM)               (None, 128)               82432     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 115617 (451.63 KB)
Trainable params: 115617 (451.63 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [12]:
results = []

# Itera sobre os folds
for train_index, test_index in kfold.split(x):
    xtrain, xtest = x[train_index], x[test_index]
    ytrain, ytest = y[train_index], y[test_index]

    history = model.fit(xtrain, ytrain, epochs=10, batch_size=32, verbose=1, validation_data=(xtrain, ytrain), callbacks=[early_stopping])

    # Avalie o modelo no conjunto de teste para o fold atual
    loss, accuracy = model.evaluate(xtest, ytest, verbose=2, batch_size=32)
    results.append(accuracy)

# Calcula a média das métricas de avaliação
mean_accuracy = np.mean(results)
print("Mean cross-validated accuracy:", mean_accuracy)


Epoch 1/10


/home/chocomenta/anaconda3/lib/python3.11/site-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


154/169 [==========================>...] - ETA: 22s - loss: 0.0000e+00 - accuracy: 0.3322

### Model n-gram3

In [ ]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df['ngram3'].values)
x = tokenizer.texts_to_sequences(df['ngram3'].values)
x = pad_sequences(x)

In [ ]:
embed_dim = 32
lstm_out = 32

model = Sequential([
    layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
    layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.LSTM(128, dropout=0.1, recurrent_dropout=0.1),
    layers.Dense(1, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
results = []

# Itera sobre os folds
for train_index, test_index in kfold.split(x):
    xtrain, xtest = x[train_index], x[test_index]
    ytrain, ytest = y[train_index], y[test_index]

    history = model.fit(xtrain, ytrain, epochs=20, batch_size=32, verbose=1, validation_data=(xtrain, ytrain), callbacks=[early_stopping])

    # Avalie o modelo no conjunto de teste para o fold atual
    loss, accuracy = model.evaluate(xtest, ytest, verbose=2, batch_size=32)
    results.append(accuracy)

# Calcula a média das métricas de avaliação
mean_accuracy = np.mean(results)
print("Mean cross-validated accuracy:", mean_accuracy)


### Model PoS

In [ ]:
df_pos = pd.read_csv('./DfPos.csv')

In [ ]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df_pos['pos'].values)
x = tokenizer.texts_to_sequences(df_pos['pos'].values)
x = pad_sequences(x)

In [ ]:
embed_dim = 32
lstm_out = 32

model = Sequential([
    layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
    layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.LSTM(128, dropout=0.1, recurrent_dropout=0.1),
    layers.Dense(1, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3662, 32)          32000     
                                                                 
 conv1d_1 (Conv1D)           (None, 3662, 32)          3104      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 1831, 32)          0         
 g1D)                                                            
                                                                 
 lstm_1 (LSTM)               (None, 128)               82432     
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 117923 (460.64 KB)
Trainable params: 117923 (460.64 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [ ]:
results = []

# Itera sobre os folds
for train_index, test_index in kfold.split(x):
    xtrain, xtest = x[train_index], x[test_index]
    ytrain, ytest = y[train_index], y[test_index]

    history = model.fit(xtrain, ytrain, epochs=10, batch_size=32, verbose=1, validation_data=(xtrain, ytrain), callbacks=[early_stopping])

    # Avalie o modelo no conjunto de teste para o fold atual
    loss, accuracy = model.evaluate(xtest, ytest, verbose=2, batch_size=32)
    results.append(accuracy)
    print(accuracy)

# Calcula a média das métricas de avaliação
mean_accuracy = np.mean(results)
print("Mean cross-validated accuracy:", mean_accuracy)


Epoch 1/10
169/169 [==============================] - 327s 2s/step - loss: 1.0687 - accuracy: 0.3978 - val_loss: 0.9793 - val_accuracy: 0.4987
Epoch 2/10
 87/169 [==============>...............] - ETA: 2:24 - loss: 0.9872 - accuracy: 0.4799

KeyboardInterrupt: 